svm_model = SVC(kernel='rbf', C=10.0, gamma='scale', probability=True)
knn_model = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')


In [1]:
# ============== LANGKAH 1: INISIALISASI & SETUP =============
import os
import sys
import glob
import cv2
import numpy as np
import json
import time
import pickle
import torch
from collections import defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from deepface import DeepFace # Diperlukan untuk get_embedding gallery
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, recall_score, f1_score, precision_score
import seaborn as sns

print("Semua pustaka berhasil diimpor.")

# Definisikan kelas Encoder kustom (jika diperlukan untuk menyimpan hasil baru)
class NumpyJSONEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        elif isinstance(obj, np.floating): return float(obj)
        elif isinstance(obj, np.ndarray): return obj.tolist()
        elif isinstance(obj, np.bool_): return bool(obj)
        else: return super(NumpyJSONEncoder, self).default(obj)

# --- Konfigurasi Path ---
BASE_DIR = os.path.abspath('.')
GALLERY_PATH = os.path.join(BASE_DIR, 'data', 'gallery6.2') # Path galeri tetap diperlukan
RESULTS_PATH = os.path.join(BASE_DIR, 'results_v6.4.3.2.1_recognition') # Path output analisis
CACHE_PATH = os.path.join(BASE_DIR, 'cache_v6.4.3.2.1_recognition') # Path cache jika diperlukan
# >>>>> PERUBAHAN PATH MODEL BARU <<<<<
NEW_MODELS_PATH = os.path.join(BASE_DIR, 'models_v6.4.3.2.1_tuned') # Path BARU untuk menyimpan model hasil tuning
PROBE_FEATURES_PATH = os.path.join(BASE_DIR, 'features_v6.4') # Path fitur probe dari v6.4

# Pastikan semua direktori ada
for path in [RESULTS_PATH, CACHE_PATH, NEW_MODELS_PATH]: # Tambahkan NEW_MODELS_PATH
    os.makedirs(path, exist_ok=True)
print("Direktori telah disiapkan.")

# --- Setup Device ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Menggunakan device: {DEVICE}')

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


Semua pustaka berhasil diimpor.
Direktori telah disiapkan.
Menggunakan device: cpu


In [2]:
# ============== LANGKAH 2: DEFINISI FUNGSI UTILITAS =============

# Fungsi get_embedding diperlukan untuk melatih classifier dari galeri
def get_embedding(image_path_or_array, model_name='ArcFace', detector_backend='retinaface') -> list | None:
    """Mengekstrak embedding wajah dari path gambar atau array (dengan deteksi)."""
    try:
        embedding_objs = DeepFace.represent(
            img_path=image_path_or_array, model_name=model_name,
            enforce_detection=True, detector_backend=detector_backend
        )
        if embedding_objs and isinstance(embedding_objs, list):
            return embedding_objs[0]['embedding']
        return None
    except Exception as e:
        # print(f"Error saat get_embedding: {e}") # Uncomment untuk debug
        return None

# Fungsi Cosine Similarity diperlukan untuk perbandingan
def cosine_similarity_prediction(query_embedding, gallery_embeddings, gallery_labels, threshold=0.5):
    """Prediksi menggunakan cosine similarity."""
    if query_embedding is None or not gallery_embeddings:
        return "unknown", 0.0, False

    query_embedding = np.array(query_embedding)
    query_norm = np.linalg.norm(query_embedding)
    if query_norm == 0:
        return "unknown", 0.0, False
    query_embedding = query_embedding / query_norm

    similarities = []
    valid_gallery_labels = []
    for gallery_emb, label in zip(gallery_embeddings, gallery_labels):
        gallery_emb = np.array(gallery_emb)
        gallery_norm = np.linalg.norm(gallery_emb)
        if gallery_norm > 0:
            gallery_emb = gallery_emb / gallery_norm
            similarity = np.dot(query_embedding, gallery_emb)
            similarities.append(similarity)
            valid_gallery_labels.append(label)

    if not similarities:
        return "unknown", 0.0, False

    max_similarity = np.max(similarities)
    max_idx = np.argmax(similarities)
    predicted_label = valid_gallery_labels[max_idx]
    is_recognized = max_similarity > threshold

    if np.isnan(max_similarity):
         return "unknown", 0.0, False

    return predicted_label, float(max_similarity), bool(is_recognized)

def cosine_similarity_top_n(query_embedding, gallery_embeddings, gallery_labels, top_n=5):
    """
    Mengembalikan Top-N label beserta skor similarity-nya.
    """
    if query_embedding is None or not gallery_embeddings:
        return [], False

    query_embedding = np.array(query_embedding)
    query_norm = np.linalg.norm(query_embedding)
    if query_norm == 0:
        return [], False
    query_embedding = query_embedding / query_norm

    # Hitung similarity untuk semua kandidat di galeri
    scores = []
    for gallery_emb, label in zip(gallery_embeddings, gallery_labels):
        gallery_emb = np.array(gallery_emb)
        gallery_norm = np.linalg.norm(gallery_emb)
        if gallery_norm > 0:
            gallery_emb = gallery_emb / gallery_norm
            sim = np.dot(query_embedding, gallery_emb)
            scores.append((label, sim))

    # Urutkan dari similarity terbesar ke terkecil
    scores.sort(key=lambda x: x[1], reverse=True)
    
    # Ambil Top-N
    # Catatan: Ini logic "Nearest Neighbor". Jika ada banyak foto 'subject_a' di galeri,
    # Top-5 bisa jadi ['a', 'a', 'a', 'b', 'c'].
    # Jika ground_truth 'a' ada di list ini, maka hitungannya benar.
    top_results = scores[:top_n]
    
    return top_results


print("Fungsi-fungsi utilitas siap digunakan.")

Fungsi-fungsi utilitas siap digunakan.


In [3]:
# ============== LANGKAH 3 (FIXED: PYIQA UNTUK NIQE & BRISQUE) =============
import pyiqa
import torch
import cv2
import numpy as np
import os
from PIL import Image

# Setup Device (Gunakan CUDA agar cepat)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running Quality Assessment on: {DEVICE}")

# Inisialisasi Model (Hanya sekali load agar tidak berat)
try:
    # Menggunakan PyIQA - Standar Penelitian CV
    print("Memuat model BRISQUE & NIQE... (Download otomatis jika belum ada)")
    brisque_metric = pyiqa.create_metric('brisque', device=DEVICE, as_loss=False)
    niqe_metric = pyiqa.create_metric('niqe', device=DEVICE, as_loss=False)
    print("Model Kualitas Citra Siap.")
except Exception as e:
    print(f"ERROR Inisialisasi PyIQA: {e}")
    brisque_metric = None
    niqe_metric = None

def calculate_image_quality(image_path):
    """
    Menghitung skor BRISQUE & NIQE menggunakan PyIQA.
    Input: Path gambar (String)
    Output: Dictionary skor
    """
    metrics = {'brisque': np.nan, 'niqe': np.nan}
    
    if not os.path.exists(image_path):
        return metrics
        
    try:
        # PyIQA membutuhkan input Tensor (RGB, 0-1, bentuk BCHW)
        # Kita pakai PIL untuk baca gambar agar formatnya benar (RGB)
        img_pil = Image.open(image_path).convert('RGB')
        
        # Konversi ke Tensor
        # PyIQA biasanya punya helper, tapi kita pakai manual agar aman:
        img_tensor = torch.from_numpy(np.array(img_pil)).float() / 255.0
        img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0).to(DEVICE)
        
        # 1. Hitung BRISQUE (Lower is Better)
        if brisque_metric is not None:
            with torch.no_grad():
                score = brisque_metric(img_tensor)
                metrics['brisque'] = score.item()
                
        # 2. Hitung NIQE (Lower is Better)
        if niqe_metric is not None:
            with torch.no_grad():
                score = niqe_metric(img_tensor)
                metrics['niqe'] = score.item()
                
    except Exception as e:
        # print(f"Error calculating metrics for {image_path}: {e}")
        pass
        
    return metrics

print("Fungsi perhitungan BRISQUE & NIQE (PyIQA) siap digunakan.")

Running Quality Assessment on: cpu
Memuat model BRISQUE & NIQE... (Download otomatis jika belum ada)
Model Kualitas Citra Siap.
Fungsi perhitungan BRISQUE & NIQE (PyIQA) siap digunakan.


In [4]:
# ============== TAMBAHAN: FUNGSI KUALITAS CITRA (BRISQUE & NIQE) =============
from brisque import BRISQUE
import cv2
import numpy as np
import os

# Inisialisasi obyek BRISQUE
# Note: Jika ini pertama kali dijalankan, ia mungkin akan mendownload model kecil
try:
    brisque_obj = BRISQUE(url=False)
except Exception as e:
    print(f"Warning: Gagal inisialisasi BRISQUE. Pastikan 'pip install brisque' sukses. Error: {e}")
    brisque_obj = None

def calculate_blur_score(image):
    """
    Menghitung skor ketajaman menggunakan Variance of Laplacian.
    Makin TINGGI nilainya, makin TAJAM gambarnya.
    Makin RENDAH (< 100), makin BLUR gambarnya.
    """
    if image is None: return 0.0
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    score = cv2.Laplacian(gray, cv2.CV_64F).var()
    return score

def calculate_image_quality(image_path):
    """
    Menghitung skor BRISQUE (Naturalness) dan Blur Score (Sharpness).
    """
    metrics = {'brisque': np.nan, 'blur_score': np.nan}
    
    try:
        if not os.path.exists(image_path):
            return metrics
            
        img = cv2.imread(image_path)
        if img is None:
            return metrics
            
        # 1. Hitung BRISQUE (Lower is Better: 0=Best, 100=Worst)
        if brisque_obj is not None:
            try:
                metrics['brisque'] = brisque_obj.score(img)
            except Exception:
                metrics['brisque'] = np.nan
        
        # 2. Hitung Blur Score / Sharpness (Higher is Better)
        # Ini menggantikan NIQE. Sangat bagus untuk analisis jarak jauh.
        try:
            metrics['blur_score'] = calculate_blur_score(img)
        except Exception:
            metrics['blur_score'] = np.nan
            
    except Exception as e:
        pass
        
    return metrics

print("Fungsi perhitungan BRISQUE & Blur Score siap digunakan.")

Fungsi perhitungan BRISQUE & Blur Score siap digunakan.


In [5]:
import os
import json
import random

# Path folder Anda (dari screenshot)
DIR_ORIGINAL = r"D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\results_v6.4_extraction\cropped_faces"
DIR_RESTORED = r"D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\results_v6.4_extraction\restored_faces"

# Path JSON
PROBE_FEATURES_PATH = r"D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\features_v6.4"
features_file = os.path.join(PROBE_FEATURES_PATH, 'probe_features_v6.4.json')

# 1. Cek isi folder fisik
files_in_folder = os.listdir(DIR_ORIGINAL)
print(f"--- ISI FOLDER REAL ({len(files_in_folder)} file) ---")
print(f"Contoh 3 file di folder: {files_in_folder[:3]}")

# 2. Cek isi JSON
with open(features_file, 'r') as f:
    data = json.load(f)

print(f"\n--- ISI JSON ({len(data)} data) ---")
sample_json = data[0]['file']
print(f"Contoh path di JSON (Raw): {sample_json}")
print(f"Contoh filename di JSON (Base): {os.path.basename(sample_json)}")

# 3. Cek Kecocokan
json_filename = os.path.basename(sample_json)
if json_filename in files_in_folder:
    print(f"\n[OK] File '{json_filename}' DITEMUKAN di folder!")
else:
    print(f"\n[ERROR] File '{json_filename}' TIDAK DITEMUKAN di folder.")
    print("Kemungkinan nama file beda atau ekstensi beda.")

--- ISI FOLDER REAL (1364 file) ---
Contoh 3 file di folder: ['a_gp_0_ef_00.jpg', 'a_gp_0_ef_01.jpg', 'a_gp_0_ef_02.jpg']

--- ISI JSON (1364 data) ---
Contoh path di JSON (Raw): a_gp_0_ef_00.jpg
Contoh filename di JSON (Base): a_gp_0_ef_00.jpg

[OK] File 'a_gp_0_ef_00.jpg' DITEMUKAN di folder!


In [11]:
# ============== LANGKAH 4 (FINAL FIX): REKOGNISI LENGKAP (KNN, SVM, COSINE) + KUALITAS =============
import torch
import cv2
import numpy as np
import os
import json
import pickle
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. KONFIGURASI PATH ---
BASE_IMG_DIR = r"D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\results_v6.4_extraction"
DIR_ORIGINAL = os.path.join(BASE_IMG_DIR, "cropped_faces")
DIR_RESTORED = os.path.join(BASE_IMG_DIR, "restored_faces")
NEW_MODELS_PATH = os.path.join(BASE_DIR, 'models_v6.4.3.2_tuned')
CACHE_PATH = os.path.join(BASE_DIR, 'cache_v6.4.3.2_recognition')

# --- 2. RELOAD SEMUA MODEL (KNN, SVM, LE, GALLERY) ---
print("Memuat ulang semua model...")
try:
    with open(os.path.join(NEW_MODELS_PATH, 'knn_model_tuned.pkl'), 'rb') as f: knn_model = pickle.load(f)
    with open(os.path.join(NEW_MODELS_PATH, 'svm_model_tuned.pkl'), 'rb') as f: svm_model = pickle.load(f)
    with open(os.path.join(NEW_MODELS_PATH, 'label_encoder.pkl'), 'rb') as f: le = pickle.load(f)
    print("[OK] KNN, SVM, LabelEncoder dimuat.")
    
    # Load Gallery untuk Cosine
    with open(os.path.join(CACHE_PATH, 'gallery_embeddings_cache.pkl'), 'rb') as f:
        gallery_embeddings, gallery_labels = pickle.load(f)
    print(f"[OK] Gallery Embeddings dimuat ({len(gallery_embeddings)} data).")
except Exception as e:
    print(f"[CRITICAL ERROR] Gagal load model: {e}")
    raise e

# --- 3. SETUP MODEL KUALITAS (BRISQUE & NIQE) ---
try:
    from brisque import BRISQUE
    brisque_obj = BRISQUE(url=False)
except: brisque_obj = None

try:
    import pyiqa
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    niqe_metric = pyiqa.create_metric('niqe', device=DEVICE, as_loss=False)
except: niqe_metric = None

# --- 4. INDEXING GAMBAR (Agar Cepat) ---
print("Mengindeks file gambar...")
orig_files_map = {f: os.path.join(DIR_ORIGINAL, f) for f in os.listdir(DIR_ORIGINAL) if f.lower().endswith(('.jpg', '.png'))}
rest_files_map = {f: os.path.join(DIR_RESTORED, f) for f in os.listdir(DIR_RESTORED) if f.lower().endswith(('.jpg', '.png'))}

# --- 5. FUNGSI HITUNG KUALITAS ---
def get_hybrid_scores(path):
    res = {'brisque': np.nan, 'niqe': np.nan}
    if not path or not os.path.exists(path): return res
    # BRISQUE
    if brisque_obj:
        try:
            img = cv2.imread(path)
            if img is not None: res['brisque'] = brisque_obj.score(img)
        except: pass
    # NIQE
    if niqe_metric:
        try:
            img_pil = Image.open(path).convert('RGB')
            t = torch.from_numpy(np.array(img_pil)).float() / 255.0
            t = t.permute(2, 0, 1).unsqueeze(0).to(DEVICE)
            with torch.no_grad(): res['niqe'] = niqe_metric(t).item()
        except: pass
    return res

# --- 6. FUNGSI PREDIKSI KOMPLIT (KNN, SVM, COSINE) ---
def predict_all_algorithms(embedding, ground_truth):
    # Default Result jika embedding kosong
    res = {
        'knn_top1': False, 'prediction_knn': 'unknown',
        'svm_top1': False, 'prediction_svm': 'unknown',
        'cosine_top1': False, 'prediction_cosine': 'unknown',
        'embedding_found': False
    }
    
    if not isinstance(embedding, list) or len(embedding) == 0: return res
    res['embedding_found'] = True
    
    try:
        emb_np = np.array(embedding).reshape(1, -1)
        
        # A. KNN
        pred_idx = knn_model.predict(emb_np)[0]
        pred_label = le.inverse_transform([pred_idx])[0]
        res['prediction_knn'] = pred_label
        res['knn_top1'] = (pred_label == ground_truth)
        
        # B. SVM
        pred_idx_svm = svm_model.predict(emb_np)[0]
        pred_label_svm = le.inverse_transform([pred_idx_svm])[0]
        res['prediction_svm'] = pred_label_svm
        res['svm_top1'] = (pred_label_svm == ground_truth)
        
        # C. COSINE SIMILARITY
        # Hitung similarity dengan semua gallery
        sims = cosine_similarity(emb_np, np.array(gallery_embeddings))[0]
        best_idx = np.argmax(sims)
        pred_label_cos = gallery_labels[best_idx]
        res['prediction_cosine'] = pred_label_cos
        res['cosine_top1'] = (pred_label_cos == ground_truth)
        
    except Exception as e:
        # print(f"Error prediction: {e}")
        pass
        
    return res

# --- 7. LOOP UTAMA ---
features_path = os.path.join(PROBE_FEATURES_PATH, 'probe_features_v6.4.json')
with open(features_path, 'r') as f: probe_features = json.load(f)

recognition_results = []

print("\nMemulai Analisis Lengkap...")
for entry in tqdm(probe_features):
    fname = os.path.basename(entry.get('file', ''))
    ground_truth = entry.get('ground_truth', 'unknown')
    
    # 1. Cari File & Hitung Quality
    p_orig = orig_files_map.get(fname)
    p_rest = rest_files_map.get(fname)
    
    s_orig = get_hybrid_scores(p_orig)
    s_rest = get_hybrid_scores(p_rest)
    
    # 2. Prediksi Semua Algoritma
    res_orig = predict_all_algorithms(entry.get('embedding_original'), ground_truth)
    res_rest = predict_all_algorithms(entry.get('embedding_restored'), ground_truth)
    
    # 3. Susun Hasil Akhir (Flatten dictionary agar mudah dibaca pandas)
    final_res = {
        'file': fname,
        'metadata': entry.get('metadata', {}),
        'ground_truth': ground_truth,
        
        # Quality
        'brisque_original': s_orig['brisque'], 'niqe_original': s_orig['niqe'],
        'brisque_restored': s_rest['brisque'], 'niqe_restored': s_rest['niqe'],
        
        # Original Predictions
        'embedding_original_found': res_orig['embedding_found'],
        'knn_top1_original': res_orig['knn_top1'],
        'prediction_knn_original': res_orig['prediction_knn'], # <--- INI YG HILANG TADI
        'svm_top1_original': res_orig['svm_top1'],
        'prediction_svm_original': res_orig['prediction_svm'],
        'cosine_top1_original': res_orig['cosine_top1'],
        'prediction_cosine_original': res_orig['prediction_cosine'],
        
        # Restored Predictions
        'embedding_restored_found': res_rest['embedding_found'],
        'knn_top1_restored': res_rest['knn_top1'],
        'prediction_knn_restored': res_rest['prediction_knn'], # <--- INI JUGA
        'svm_top1_restored': res_rest['svm_top1'],
        'prediction_svm_restored': res_rest['prediction_svm'],
        'cosine_top1_restored': res_rest['cosine_top1'],
        'prediction_cosine_restored': res_rest['prediction_cosine']
    }
    recognition_results.append(final_res)

print("Selesai. Data siap untuk Langkah 5.")

Memuat ulang semua model...
[OK] KNN, SVM, LabelEncoder dimuat.
[OK] Gallery Embeddings dimuat (55 data).
Mengindeks file gambar...

Memulai Analisis Lengkap...


  0%|          | 0/1364 [00:00<?, ?it/s]

Selesai. Data siap untuk Langkah 5.


In [12]:
# ============== LANGKAH 5 (FINAL): ANALISIS PERBANDINGAN AKURASI vs KUALITAS CITRA =============
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

# Pastikan ada data
if not recognition_results:
    print("\n[WARNING] Tidak ada hasil rekognisi untuk dianalisis.")
else:
    df_results = pd.DataFrame(recognition_results)
    
    print(f"\n{'='*100}")
    print("ANALISIS PERBANDINGAN: KUALITAS CITRA (BRISQUE/NIQE) vs AKURASI REKOGNISI".center(100))
    print(f"{'='*100}\n")

    # Filter Data Valid (yang punya embedding)
    df_original = df_results[df_results['embedding_original_found'] == True].copy()
    df_restored = df_results[df_results['embedding_restored_found'] == True].copy()

    # Ekstrak Metadata Jarak & Tinggi
    for df in [df_original, df_restored]:
        if not df.empty:
            df['distance_m'] = df['metadata'].apply(lambda x: x.get('distance_m'))
            df['height_m'] = df['metadata'].apply(lambda x: x.get('height_m'))

    # --- FUNGSI HELPER HITUNG METRIK ---
    def get_summary_stats(df, suffix, model_key, labels_list):
        stats = {}
        
        # 1. Hitung Rata-rata Kualitas Citra (Tahan terhadap NaN)
        stats['avg_brisque'] = df[f'brisque{suffix}'].mean() if f'brisque{suffix}' in df.columns else 0
        stats['avg_niqe'] = df[f'niqe{suffix}'].mean() if f'niqe{suffix}' in df.columns else 0
        
        # 2. Hitung Metrik Rekognisi
        col_pred = f'prediction_{model_key}{suffix}'
        # Logic Top-1 Accuracy (sesuai output Step 4 sebelumnya yang menyimpan boolean)
        col_correct = f'{model_key}_top1{suffix}' 
        
        if col_correct in df.columns:
            stats['accuracy'] = df[col_correct].mean()
            
            # Hitung F1 Score Macro
            y_true = df['ground_truth']
            y_pred = df[col_pred]
            stats['f1'] = f1_score(y_true, y_pred, average='macro', zero_division=0, labels=labels_list)
        else:
            stats['accuracy'] = 0
            stats['f1'] = 0
            
        return stats

    # --- DEFINISI SKENARIO PENGUJIAN ---
    scenarios = [
        {"name": "Jarak Dekat (< 7m)", "col": "distance_m", "cond": lambda x: x < 7},
        {"name": "Jarak Menengah (7-12m)", "col": "distance_m", "cond": lambda x: 7 <= x < 12},
        {"name": "Jarak Jauh (>= 12m)", "col": "distance_m", "cond": lambda x: x >= 12},
        {"name": "Ketinggian 1.5m", "col": "height_m", "cond": lambda x: x == 1.5},
        {"name": "Ketinggian 3m", "col": "height_m", "cond": lambda x: x == 3},
        {"name": "Ketinggian 4m", "col": "height_m", "cond": lambda x: x == 4},
        {"name": "Ketinggian 5m", "col": "height_m", "cond": lambda x: x == 5},
        {"name": "Semua Data Uji", "col": None, "cond": None}
    ]

    models = [
        ('knn', 'K-Nearest Neighbors (KNN)'),
        ('svm', 'Support Vector Machine (SVM)'),
        ('cosine', 'Cosine Similarity')
    ]

    # --- LOOP UTAMA PEMBUATAN TABEL ---
    for model_key, model_name in models:
        table_rows = []
        
        for sc in scenarios:
            # Filter Data per Skenario
            if sc['col']:
                d_orig = df_original[df_original[sc['col']].apply(sc['cond'])]
                d_rest = df_restored[df_restored[sc['col']].apply(sc['cond'])]
            else:
                d_orig = df_original
                d_rest = df_restored
            
            # Ambil semua label unik di skenario ini untuk perhitungan F1
            all_labels = sorted(set(d_orig['ground_truth'].unique()) | set(d_rest['ground_truth'].unique()))
            
            # --- BARIS 1: ORIGINAL ---
            if not d_orig.empty:
                s = get_summary_stats(d_orig, '_original', model_key, all_labels)
                table_rows.append({
                    'Skenario': sc['name'],
                    'Metode': 'Original',
                    'Avg BRISQUE': s['avg_brisque'],
                    'Avg NIQE': s['avg_niqe'],
                    'Akurasi': s['accuracy'],
                    'F1-Score': s['f1'],
                    'Improvement': '-'
                })
            else:
                table_rows.append({'Skenario': sc['name'], 'Metode': 'Original', 'Akurasi': 0, 'Improvement': '-'})

            # --- BARIS 2: RESTORED ---
            if not d_rest.empty:
                s_res = get_summary_stats(d_rest, '_restored', model_key, all_labels)
                
                # Hitung Peningkatan Akurasi
                imp_str = "-"
                if not d_orig.empty:
                    acc_orig = get_summary_stats(d_orig, '_original', model_key, all_labels)['accuracy']
                    if acc_orig > 0:
                        imp = ((s_res['accuracy'] - acc_orig) / acc_orig) * 100
                        imp_str = f"{imp:+.2f}%"
                
                table_rows.append({
                    'Skenario': '', # Kosongkan nama skenario biar rapi (grouped view)
                    'Metode': 'Restored',
                    'Avg BRISQUE': s_res['avg_brisque'],
                    'Avg NIQE': s_res['avg_niqe'],
                    'Akurasi': s_res['accuracy'],
                    'F1-Score': s_res['f1'],
                    'Improvement': imp_str
                })
            else:
                 table_rows.append({'Skenario': '', 'Metode': 'Restored', 'Akurasi': 0, 'Improvement': '-'})

        # --- BUAT DATAFRAME TABEL ---
        df_table = pd.DataFrame(table_rows)
        
        # Format Kolom Angka agar Cantik
        # Menggunakan .map agar tidak error jika ada data non-numeric/NaN
        df_table['Avg BRISQUE'] = df_table['Avg BRISQUE'].map(lambda x: f"{x:.2f}" if pd.notnull(x) and isinstance(x, (int, float)) else "-")
        df_table['Avg NIQE'] = df_table['Avg NIQE'].map(lambda x: f"{x:.2f}" if pd.notnull(x) and isinstance(x, (int, float)) else "-")
        df_table['Akurasi'] = df_table['Akurasi'].map(lambda x: f"{x:.2%}" if pd.notnull(x) and isinstance(x, (int, float)) else "0.00%")
        df_table['F1-Score'] = df_table['F1-Score'].map(lambda x: f"{x:.2%}" if pd.notnull(x) and isinstance(x, (int, float)) else "0.00%")
        
        # Reorder Columns
        cols = ['Skenario', 'Metode', 'Avg BRISQUE', 'Avg NIQE', 'Akurasi', 'F1-Score', 'Improvement']
        df_table = df_table[cols]
        
        # PRINT TABEL
        print(f"\n{'='*110}")
        print(f"TABEL HASIL: {model_name}".center(110))
        print(f"{'='*110}")
        print(df_table.to_string(index=False))
        
        # Simpan ke CSV
        csv_name = f"final_analysis_{model_key}_with_quality.csv"
        df_table.to_csv(os.path.join(RESULTS_PATH, csv_name), index=False)
        print(f"[Info] Tabel disimpan ke: {csv_name}")

print(f"\n{'='*100}")
print("ANALISIS SELESAI".center(100))
print(f"{'='*100}")


             ANALISIS PERBANDINGAN: KUALITAS CITRA (BRISQUE/NIQE) vs AKURASI REKOGNISI              


                                    TABEL HASIL: K-Nearest Neighbors (KNN)                                    
              Skenario   Metode Avg BRISQUE Avg NIQE Akurasi F1-Score Improvement
    Jarak Dekat (< 7m) Original        3.91     4.27  84.10%   85.09%           -
                       Restored        3.90     4.26  77.27%   79.91%      -8.12%
Jarak Menengah (7-12m) Original        1.69     4.26  56.04%   59.73%           -
                       Restored        1.70     4.26  72.95%   72.48%     +30.19%
   Jarak Jauh (>= 12m) Original        3.78     4.09  11.30%    5.48%           -
                       Restored        3.69     4.09  41.53%   39.57%    +267.61%
       Ketinggian 1.5m Original        8.81     4.53  58.82%   65.67%           -
                       Restored        8.81     4.53  65.69%   66.30%     +11.67%
         Ketinggian 3m Original        4.54    